In [ ]:
import matplotlib.pyplot as plt
import gradio as gr
import numpy as np
import plotly.express as px

In [ ]:
#demo
aspects = ['Battery', 'Service', 'Others']
aspects_ratio = (np.random.dirichlet(np.ones(3), size=1) * 100).flatten()

sentiments_ratio = (np.random.dirichlet(np.ones(3), size=1) * 100).flatten()
sentiments = ['Positive', 'Negative', 'Neutral']

aspects_polarity = []
aspects_polarity_ratio = (np.random.dirichlet(np.ones(9), size=1) * 100).flatten()
for aspect in aspects:
    for sentiment in sentiments:
        aspects_polarity.append(aspect + '#' + sentiment)    

def sentiment_analysis(text, aspect):

    # Tạo biểu đồ cảm xúc theo aspect
    pie_sentiments_of_an_aspect = draw_pie_sentiments_of_an_aspect(aspect)

    #Biểu đồ aspect
    pie_of_all_aspect = draw_pie_of_aspect()

    #Biểu đồ aspect#polirity
    pie_aspect_polarity = draw_pie_aspect_polarity()

    return (pie_sentiments_of_an_aspect, pie_of_all_aspect, pie_aspect_polarity)

def draw_pie_sentiments_of_an_aspect(aspect):
    sentiments_ratio = (np.random.dirichlet(np.ones(3), size=1) * 100).flatten()
    pie_sentiments_of_an_aspect = px.pie(values=sentiments_ratio, names=sentiments, title=aspect)
    pie_sentiments_of_an_aspect.update_traces(textinfo='percent')
    pie_sentiments_of_an_aspect.update_traces(textposition=['inside' if ratio > 5 else 'outside' for ratio in sentiments_ratio])
    pie_sentiments_of_an_aspect.update_layout(title_x=0.465)
    return pie_sentiments_of_an_aspect

def draw_pie_of_aspect():
    pie_of_aspect = px.pie(values=aspects_ratio, names=aspects, title='Aspect')
    pie_of_aspect.update_traces(textinfo='percent')
    pie_of_aspect.update_traces(textposition=['inside' if ratio > 5 else 'outside' for ratio in aspects_ratio])
    pie_of_aspect.update_layout(title_x=0.45)
    return pie_of_aspect

def draw_pie_aspect_polarity():
    pie_aspect_polarity = px.pie(values=aspects_polarity_ratio, names=aspects_polarity, title='Aspect#Polarity')
    pie_aspect_polarity.update_traces(textinfo='percent')
    pie_aspect_polarity.update_traces(textposition=[ 'inside' if ratio > 5 else 'outside' for ratio in aspects_polarity_ratio])
    pie_aspect_polarity.update_layout(legend=dict(orientation="v", yanchor="top", y=1, xanchor="right", x=1))
    pie_aspect_polarity.update_layout(title_x=0.489)
    return pie_aspect_polarity

def submit(comment, aspect):
    pie_plot = sentiment_analysis(comment, aspect)
    return {
        output_plot1: gr.Column(visible=True),
        output_plot2: gr.Column(visible=True), 
        pie_sentiment: pie_plot[0],
        pie_all_aspect: pie_plot[1],
        pie_aspect_polarity: pie_plot[2]
    }
    

with gr.Blocks() as demo:
    with gr.Row():
        text_box = gr.Textbox(placeholder="Write your comment here...", visible=True, label="Comment")
        submit_btn = gr.Button(value="Submit")
    with gr.Row(visible=False) as output_plot1:
        with gr.Column():
            with gr.Row():
                choose_aspect_dropdown = gr.Dropdown(
                    choices=['General', 'Battery', 'Service', 'Others'], 
                    label="Choose Aspect",
                    value='General',
                    )
                sentiment = ''
                # sentiment_report = gr.Button(sentiments[sentiments_ratio.argmax()])
            pie_sentiment = gr.Plot()
        pie_all_aspect = gr.Plot() 
    with gr.Row(visible=False) as output_plot2:
        pie_aspect_polarity = gr.Plot(visible=False)

    choose_aspect_dropdown.select(
            fn = draw_pie_sentiments_of_an_aspect,
            inputs = [choose_aspect_dropdown],
            outputs = [pie_sentiment],
        )

    submit_btn.click(
    fn = submit,
    inputs = [text_box, choose_aspect_dropdown],
    outputs = [output_plot1, output_plot2, pie_sentiment, pie_all_aspect, pie_aspect_polarity],
)

demo.launch(share=False)

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    error_box = gr.Textbox(label="Error", visible=False)

    name_box = gr.Textbox(label="Name")
    age_box = gr.Number(label="Age", minimum=0, maximum=100)
    symptoms_box = gr.CheckboxGroup(["Cough", "Fever", "Runny Nose"])
    submit_btn = gr.Button("Submit")

    with gr.Column(visible=False) as output_col:
        diagnosis_box = gr.Textbox(label="Diagnosis")
        patient_summary_box = gr.Textbox(label="Patient Summary")

    def submit(name, age, symptoms):
        if len(name) == 0:
            return {error_box: gr.Textbox(value="Enter name", visible=True)}
        return {
            output_col: gr.Column(visible=True),
            diagnosis_box: "covid" if "Cough" in symptoms else "flu",
            patient_summary_box: f"{name}, {age} y/o",
        }

    submit_btn.click(
        submit,
        [name_box, age_box, symptoms_box],
        [error_box, diagnosis_box, patient_summary_box, output_col],
    )

demo.launch()

In [ ]:
def preprocess(text):
    # Preprocess text (username and link placeholders)
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

def sentiment_analysis(text):
    scores = [0.1, 0.2, 0.7]

    # Map the scores to labels
    labels = ['Negative', 'Neutral', 'Positive']
    scores_dict = {label: score for label, score in zip(labels, scores)}

    return scores_dict

title = "Sentiment Analysis Application\n\n\nThis application assesses if a twitter post relating to vaccination is positive,neutral or negative"

demo = gr.Interface(
    fn=sentiment_analysis,
    inputs=gr.Textbox(placeholder="Write your tweet here..."),
    outputs=gr.Label(num_top_classes=3),
    examples=[["The Vaccine is harmful!"],["I cant believe people don't vaccinate their kids"],["FDA think just not worth the AE unfortunately"],["For a vaccine given to healthy"]],
     title=title
)

demo.launch(share=False)